In [911]:
import random


In [912]:
wuerfel = []
wuerfel.append([random.randint(1, 6) for i in range(0, 5)])
wuerfel = wuerfel[0]
print(sorted(wuerfel))


[1, 3, 5, 5, 5]


In [913]:
classCheck = [0, 0, 0, 0, 0, 0, 0, 0]
classes = (
    "P",  # Augenzahl (hier Paare um aus 3er Pasch und Paar ein Full House zu finden)
    "3P",  # 3er Pasch
    "4P",  # 4er Pasch
    "F",  # Full House
    "KS",  # Kleine Straße
    "GS",  # Große Straße
    "K",  # Kniffel
    "C",  # Chance
)
classDict = dict(zip(classes, classCheck))
print(sorted(wuerfel))


[1, 3, 5, 5, 5]


In [914]:
wuerfel.sort()
paare = []
for l in range(1, 7):

    for i in range(0, 5):  # 1. Stelle i

        for j in range(0, 5):  # 2. Stelle j
            if j != i:

                for k in range(0, 5):  # 3. Stelle k
                    if k != j and k != i:

                        # 3er Pasch ----------------------------------
                        if (
                            wuerfel[i] == wuerfel[j] == wuerfel[k]
                            and classDict["3P"] == 0
                        ):
                            # Speichert die 3er Paschzahl in classCheck ab
                            classCheck[1] = wuerfel[i]
                            # Setzt 3er Pasch von 0 auf 1 (False -> True)
                            classDict["3P"] = 1

                        # Paare für Fullhouse schauen ----------------------
                        if (
                            wuerfel[i] == wuerfel[j]  # Beide Würfel vergleichen
                            # Schauen ob bereits ein Eintrag in classDict gemacht wurde <2 weil zwei Paare möglich sind
                            and classDict["P"] < 2
                            # Schaut ob erstes gefundenes Paar ungleich dem Würfel ist
                            and classCheck[0] != wuerfel[i]
                            # Schaut ob aktueller Würfel nicht die Zahl des Pasch ist
                            and classCheck[1] != wuerfel[i]
                            # Erst ab l>2 anfangen nach Paaren zu suchen, weil sonst Paare vor Pasche gefunden werden und doppelt vorkommen
                            and l > 1
                        ):
                            # Speichert erstes paar in classCheck ab
                            classCheck[0] = wuerfel[i]
                            # Speichert Paare in Liste, damit am Ende alle Paare in classCheck eingefügt werden können
                            paare.append(wuerfel[i])
                            # classDict += weil es hierbei zwei mögliche Paare gibt und nicht nur eins wie bei 3er oder 4er Pasch
                            classDict["P"] += 1

                        for m in range(0, 5):  # 4. Stelle m

                            if m != j and m != i and m != k:
                                # 4er Pasch ----------------------------------
                                if (
                                    wuerfel[i] == wuerfel[j] == wuerfel[k] == wuerfel[m]
                                    and classDict["4P"] == 0
                                ):
                                    # Speichert die 4er Paschzahl in classCheck ab
                                    classCheck[1] = wuerfel[i]
                                    # Setzt 4er Pasch von 0 auf 1 (False -> True)
                                    classDict["4P"] = 1
                                    print(f"4er Pasch mit Zahl: {wuerfel[i]}")
                                # Kleine Straße ------------------------------
                                if (
                                    [wuerfel[i], wuerfel[j], wuerfel[k], wuerfel[m]]
                                    == [1, 2, 3, 4]
                                    or [wuerfel[i], wuerfel[j], wuerfel[k], wuerfel[m]]
                                    == [2, 3, 4, 5]
                                    or [wuerfel[i], wuerfel[j], wuerfel[k], wuerfel[m]]
                                    == [3, 4, 5, 6]
                                    and classDict["KS"] != 1
                                ):
                                    classDict["KS"] = 1
                                    
                                for n in range(0, 5):   # 5. Stelle n
                                    
                                    if n != j and n != i and n != k and n!=m:
                                        pass

# Nach Full House schauen indem Paar und Pasch im Klassen Dictionary als Wahr (also 1) angegeben sind
if classDict["3P"] == 1 and classDict["P"] >= 1:
    print("FULL HOUSE")
    classDict["F"] = 1
classCheck[0] = paare
print(
    f"Paar/e: {classCheck[0]}, " if classDict["P"] > 0 else "",
    f"3er Pasch mit Zahl: {classCheck[1]}" if classDict["3P"] == 1 else "",
    f"4er Pasch mit Zahl: {classCheck[2]}" if classDict["4P"] == 1 else "",
    f"Kleine Straße" if classDict["KS"] == 1 else "",
)



 3er Pasch mit Zahl: 5  


In [915]:
if (
    [wuerfel[0], wuerfel[1], wuerfel[2], wuerfel[3]] == [1, 2, 3, 4]
    or [wuerfel[0], wuerfel[1], wuerfel[2], wuerfel[3]] == [2, 3, 4, 5]
    or [wuerfel[0], wuerfel[1], wuerfel[2], wuerfel[3]] == [3, 4, 5, 6]
):
    print("kleine Straße")
